In [85]:
import requests as req #정적 크롤링
from bs4 import BeautifulSoup as bs # 정적, 동적 크롤링
import pandas as pd # 파이썬 버전 엑셀?

res = req.get('https://www.aladin.co.kr/shop/common/wbest.aspx?BranchType=1&BestType=Bestseller')
# https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page=1&cnt=1000&SortOrder=1

html = res.text
soup = bs(html,'lxml')

#책이름
#Myform > div:nth-child(3) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(2) > a.bo3 > b
#Myform > div:nth-child(4) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(2) > a.bo3 > b
# 바뀌는 부분 앞 myform 하위 에서 필요한 정보를 담고있는 class 찾기
# 오류 발생해서 오류가 발생한 경로 가져오기
#Myform > div:nth-child(29) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(1) > a > b
#Myform > div:nth-child(30) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(2) > a > b
#오류 발생지점은 li:nth-child(2) 부분이 다름,  주변 포함해서 a > b 로 바뀜

#저자
#Myform > div:nth-child(3) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(3)
#Myform > div:nth-child(4) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(3)

# 출판사
# 저자 범위에 같이 나오므로 인덱스 값으로 불러옴

#원가, 세일가
#Myform > div:nth-child(3) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(4)
#Myform > div:nth-child(4) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(4)

# 구분
#Myform > div:nth-child(3) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(2) > span
#Myform > div:nth-child(4) > table > tbody > tr > td:nth-child(3) > table > tbody > tr:nth-child(1) > td:nth-child(1) > div:nth-child(1) > ul > li:nth-child(2) > span

#Myform .ss_book_box
data = []
infos = soup.select('#Myform .ss_book_box')

for info in infos:
    책이름 = info.select_one('li:nth-last-child(4) b').text
    저자 = info.select_one('li:nth-last-child(3)').text.split('|')[0].strip() # 저자 출판사 출판일이 나오므로 0번 인덱스인 저자만 표시 공백 오류 제거를 위해split
    출판사 = info.select_one('li:nth-last-child(3)').text.split('|')[1].strip()
    원가 = info.select_one('li:nth-last-child(2) span').text.replace(',','')
    세일가 = info.select_one('li:nth-last-child(2) .ss_p2 span').text.replace(',','')
    구분 = info.select_one('li:nth-last-child(4) > span').text.replace('[','').replace(']','')
    print(책이름, 저자, 출판사, 원가, 세일가, 구분, sep = '|')
    data.append({'도서명':책이름,'저자':저자,'출판사':출판사,'원가':원가,'세일가':세일가,'구분':구분})

df = pd.DataFrame(data)
df.set_index('도서명',inplace=True)
df.to_csv('알라딘주간베스트_utf-8.csv',encoding='utf-8')
df.to_csv('알라딘주간베스트_utf-8-sig.csv',encoding='utf-8-sig')
df.to_csv('알라딘주간베스트_euc-kr.csv',encoding='euc-kr')

이중 하나는 거짓말|김애란 (지은이)|문학동네|16000|14400|국내도서
영원한 천국|정유정 (지은이)|은행나무|19800|17820|국내도서
만화로 보는 3분 철학 : 서양 고대 철학편|김재훈, 서정욱 (지은이)|카시오페아|16000|14400|국내도서
사피엔스 : 그래픽 히스토리 Vol.1|유발 하라리 (원작), 다니엘 카사나브 (그림), 김명주 (옮긴이), 다비드 반데르묄렝 (각색)|김영사|17800|16020|국내도서
김켈리의 신비마트 1|이세경 (지은이), 김켈리 (원작), 박상빈 (그림), 박제련 (채색)|베가북스|16000|14400|국내도서
명탐정 코난 105 (특장판)|아오야마 고쇼 (지은이)|서울미디어코믹스(서울문화사)|10000|9000|국내도서
복복깜짝북|미공개 (지은이)|복복서가|18500|16650|국내도서
원펀맨 One Punch Man 30|ONE (지은이), 무라타 유스케 (그림)|대원씨아이(만화)|5500|4950|국내도서
당신이 누군가를 죽였다|히가시노 게이고 (지은이), 최고은 (옮긴이)|북다|19800|17820|국내도서
너에게 들려주는 단단한 말|김종원 (지은이)|퍼스트펭귄|18000|16200|국내도서
불안 세대|조너선 하이트 (지은이), 이충호 (옮긴이)|웅진지식하우스|24800|22320|국내도서
더 좋은 문장을 쓰고 싶은 당신을 위한 필사책|이주윤 (지은이)|빅피시|22000|19800|국내도서
푸른 들판을 걷다|클레어 키건 (지은이), 허진 (옮긴이)|다산책방|16800|15120|국내도서
저속노화 식사법|정희원 (지은이)|테이스트북스|19000|17100|국내도서
ETS 토익 정기시험 기출문제집 1000 Vol. 4 Reading (리딩)|ETS (엮은이)|(주)YBM(와이비엠)|19800|17820|국내도서
명탐정 코난 105|아오야마 고쇼 (지은이)|서울미디어코믹스(서울문화사)|6000|5400|국내도서
ETS 토익 정기시험 기출문제집 1000 Vol. 4 Listening (리스닝)|ETS (엮은이)|

In [103]:
import requests as req
from bs4 import BeautifulSoup as bs 
import pandas as pd 

data = []

for i in range(1,21):
    res = req.get(f'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page={i}&cnt=1000&SortOrder=1')
    html = res.text
    soup = bs(html,'lxml')
    infos = soup.select('#Myform .ss_book_box')

    for info in infos:
        try:
            책이름 = info.select_one('li:nth-last-child(4) b').text
            저자 = info.select_one('li:nth-last-child(3)').text.split('|')[0].strip() # 저자 출판사 출판일이 나오므로 0번 인덱스인 저자만 표시 공백 오류 제거를 위해split
            출판사 = info.select_one('li:nth-last-child(3)').text.split('|')[1].strip()
            원가 = info.select_one('li:nth-last-child(2) span').text.replace(',','')
            세일가 = info.select_one('li:nth-last-child(2) .ss_p2 span').text.replace(',','')
            구분 = info.select_one('li:nth-last-child(4) > span').text.replace('[','').replace(']','')
            
        except:
            책이름 = info.select_one('li:nth-last-child(3) b').text
            저자 = info.select_one('li:nth-last-child(2)').text.split('|')[0].strip() # 저자 출판사 출판일이 나오므로 0번 인덱스인 저자만 표시 공백 오류 제거를 위해split
            출판사 = info.select_one('li:nth-last-child(2)').text.split('|')[1].strip()
            원가 = info.select_one('li:nth-last-child(1) span').text.replace(',','')
            세일가 = info.select_one('li:nth-last-child(1) .ss_p2 span').text.replace(',','')
            구분 = info.select_one('li:nth-last-child(3) > span').text.replace('[','').replace(']','')
        print(책이름, 저자, 출판사, 원가, 세일가, 구분, sep = '|')
        data.append({'도서명':책이름,'저자':저자,'출판사':출판사,'원가':원가,'세일가':세일가,'구분':구분})
df = pd.DataFrame(data)
df.set_index('도서명',inplace=True)
df.to_csv('알라딘주간베스트_utf-8.csv',encoding='utf-8')
df.to_csv('알라딘주간베스트_utf-8-sig.csv',encoding='utf-8-sig')
df.to_csv('알라딘주간베스트_euc-kr.csv',encoding='euc-kr')

이중 하나는 거짓말|김애란 (지은이)|문학동네|16000|14400|국내도서
영원한 천국|정유정 (지은이)|은행나무|19800|17820|국내도서
만화로 보는 3분 철학 : 서양 고대 철학편|김재훈, 서정욱 (지은이)|카시오페아|16000|14400|국내도서
사피엔스 : 그래픽 히스토리 Vol.1|유발 하라리 (원작), 다니엘 카사나브 (그림), 김명주 (옮긴이), 다비드 반데르묄렝 (각색)|김영사|17800|16020|국내도서
김켈리의 신비마트 1|이세경 (지은이), 김켈리 (원작), 박상빈 (그림), 박제련 (채색)|베가북스|16000|14400|국내도서
명탐정 코난 105 (특장판)|아오야마 고쇼 (지은이)|서울미디어코믹스(서울문화사)|10000|9000|국내도서
복복깜짝북|미공개 (지은이)|복복서가|18500|16650|국내도서
원펀맨 One Punch Man 30|ONE (지은이), 무라타 유스케 (그림)|대원씨아이(만화)|5500|4950|국내도서
당신이 누군가를 죽였다|히가시노 게이고 (지은이), 최고은 (옮긴이)|북다|19800|17820|국내도서
너에게 들려주는 단단한 말|김종원 (지은이)|퍼스트펭귄|18000|16200|국내도서
불안 세대|조너선 하이트 (지은이), 이충호 (옮긴이)|웅진지식하우스|24800|22320|국내도서
더 좋은 문장을 쓰고 싶은 당신을 위한 필사책|이주윤 (지은이)|빅피시|22000|19800|국내도서
푸른 들판을 걷다|클레어 키건 (지은이), 허진 (옮긴이)|다산책방|16800|15120|국내도서
저속노화 식사법|정희원 (지은이)|테이스트북스|19000|17100|국내도서
ETS 토익 정기시험 기출문제집 1000 Vol. 4 Reading (리딩)|ETS (엮은이)|(주)YBM(와이비엠)|19800|17820|국내도서
명탐정 코난 105|아오야마 고쇼 (지은이)|서울미디어코믹스(서울문화사)|6000|5400|국내도서
ETS 토익 정기시험 기출문제집 1000 Vol. 4 Listening (리스닝)|ETS (엮은이)|

In [114]:
# if문 사용하면서 시간복잡도 잡기
import requests as req 
from bs4 import BeautifulSoup as bs 
import pandas as pd 

data = []

for i in range(1,21):
    res = req.get(f'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page={i}&cnt=1000&SortOrder=1')
    html = res.text
    soup = bs(html,'lxml')
    infos = soup.select('#Myform .ss_book_box')

    for info in infos:
            책이름 = info.select_one('li:nth-last-child(4) b')
            저자 = ''
            출판사 = ''
            원가 = ''
            세일가 = ''
            구분 = ''
            if 책이름 != None:
                책이름 = 책이름.text.strip()
                저자 = info.select_one('li:nth-last-child(3)').text.split('|')[0].strip()
                출판사 = info.select_one('li:nth-last-child(3)').text.split('|')[1].strip()
                원가 = info.select_one('li:nth-last-child(2) span').text.replace(',','')
                세일가 = info.select_one('li:nth-last-child(2) .ss_p2 span').text.replace(',','')
                구분 = info.select_one('li:nth-last-child(4) > span').text.replace('[','').replace(']','')
                
            else:
                책이름 = info.select_one('li:nth-last-child(3) b').text
                저자 = info.select_one('li:nth-last-child(2)').text.split('|')[0].strip()
                출판사 = info.select_one('li:nth-last-child(2)').text.split('|')[1].strip()
                원가 = info.select_one('li:nth-last-child(1) span').text.replace(',','')
                세일가 = info.select_one('li:nth-last-child(1) .ss_p2 span').text.replace(',','')
                구분 = info.select_one('li:nth-last-child(3) > span').text.replace('[','').replace(']','')
            print(책이름, 저자, 출판사, 원가, 세일가, 구분, sep = '|')
            data.append({'도서명':책이름,'저자':저자,'출판사':출판사,'원가':원가,'세일가':세일가,'구분':구분})
df = pd.DataFrame(data)
# df.set_index('도서명',inplace=True)
# df.to_csv('알라딘주간베스트_utf-8.csv',encoding='utf-8')
# df.to_csv('알라딘주간베스트_utf-8-sig.csv',encoding='utf-8-sig')
# df.to_csv('알라딘주간베스트_euc-kr.csv',encoding='euc-kr')

이중 하나는 거짓말|김애란 (지은이)|문학동네|16000|14400|국내도서
영원한 천국|정유정 (지은이)|은행나무|19800|17820|국내도서
만화로 보는 3분 철학 : 서양 고대 철학편|김재훈, 서정욱 (지은이)|카시오페아|16000|14400|국내도서
사피엔스 : 그래픽 히스토리 Vol.1|유발 하라리 (원작), 다니엘 카사나브 (그림), 김명주 (옮긴이), 다비드 반데르묄렝 (각색)|김영사|17800|16020|국내도서
김켈리의 신비마트 1|이세경 (지은이), 김켈리 (원작), 박상빈 (그림), 박제련 (채색)|베가북스|16000|14400|국내도서
명탐정 코난 105 (특장판)|아오야마 고쇼 (지은이)|서울미디어코믹스(서울문화사)|10000|9000|국내도서
복복깜짝북|미공개 (지은이)|복복서가|18500|16650|국내도서
원펀맨 One Punch Man 30|ONE (지은이), 무라타 유스케 (그림)|대원씨아이(만화)|5500|4950|국내도서
당신이 누군가를 죽였다|히가시노 게이고 (지은이), 최고은 (옮긴이)|북다|19800|17820|국내도서
너에게 들려주는 단단한 말|김종원 (지은이)|퍼스트펭귄|18000|16200|국내도서
불안 세대|조너선 하이트 (지은이), 이충호 (옮긴이)|웅진지식하우스|24800|22320|국내도서
더 좋은 문장을 쓰고 싶은 당신을 위한 필사책|이주윤 (지은이)|빅피시|22000|19800|국내도서
푸른 들판을 걷다|클레어 키건 (지은이), 허진 (옮긴이)|다산책방|16800|15120|국내도서
저속노화 식사법|정희원 (지은이)|테이스트북스|19000|17100|국내도서
ETS 토익 정기시험 기출문제집 1000 Vol. 4 Reading (리딩)|ETS (엮은이)|(주)YBM(와이비엠)|19800|17820|국내도서
명탐정 코난 105|아오야마 고쇼 (지은이)|서울미디어코믹스(서울문화사)|6000|5400|국내도서
ETS 토익 정기시험 기출문제집 1000 Vol. 4 Listening (리스닝)|ETS (엮은이)|

In [139]:
# 2. try~except 구문으로 구현하기
import requests as req 
from bs4 import BeautifulSoup as bs 
import pandas as pd 

data = []

for i in range(1,21):
    res = req.get(f'https://www.aladin.co.kr/shop/common/wbest.aspx?BestType=Bestseller&BranchType=1&CID=0&page={i}&cnt=1000&SortOrder=1')
    html = res.text
    soup = bs(html,'lxml')
    infos = soup.select('#Myform .ss_book_box')

    for info in infos:
        try:
            책이름 = info.select_one('li:nth-last-child(4) b').text.strip()

        except:
            책이름 = info.select_one('li:nth-last-child(3) b').text
            저자 = info.select_one('li:nth-last-child(2)').text.split('|')[0].strip()
            출판사 = info.select_one('li:nth-last-child(2)').text.split('|')[1].strip()
            원가 = info.select_one('li:nth-last-child(1) span').text.replace(',','')
            세일가 = info.select_one('li:nth-last-child(1) .ss_p2 span').text.replace(',','')
            구분 = info.select_one('li:nth-last-child(3) > span').text.replace('[','').replace(']','')
        else:
            저자 = info.select_one('li:nth-last-child(3)').text.split('|')[0].strip()
            출판사 = info.select_one('li:nth-last-child(3)').text.split('|')[1].strip()
            원가 = info.select_one('li:nth-last-child(2) span').text.replace(',','')
            세일가 = info.select_one('li:nth-last-child(2) .ss_p2 span').text.replace(',','')
            구분 = info.select_one('li:nth-last-child(4) > span').text.replace('[','').replace(']','')
        print(책이름, 저자, 출판사, 원가, 세일가, 구분, sep = '|')
        data.append({'도서명':책이름,'저자':저자,'출판사':출판사,'원가':원가,'세일가':세일가,'구분':구분})
df = pd.DataFrame(data)
df.set_index('도서명',inplace=True)
df.to_csv('알라딘주간베스트_utf-8.csv',encoding='utf-8')
df.to_csv('알라딘주간베스트_utf-8-sig.csv',encoding='utf-8-sig')
df.to_csv('알라딘주간베스트_euc-kr.csv',encoding='euc-kr')

이중 하나는 거짓말|김애란 (지은이)|문학동네|16000|14400|국내도서
영원한 천국|정유정 (지은이)|은행나무|19800|17820|국내도서
만화로 보는 3분 철학 : 서양 고대 철학편|김재훈, 서정욱 (지은이)|카시오페아|16000|14400|국내도서
사피엔스 : 그래픽 히스토리 Vol.1|유발 하라리 (원작), 다니엘 카사나브 (그림), 김명주 (옮긴이), 다비드 반데르묄렝 (각색)|김영사|17800|16020|국내도서
김켈리의 신비마트 1|이세경 (지은이), 김켈리 (원작), 박상빈 (그림), 박제련 (채색)|베가북스|16000|14400|국내도서
명탐정 코난 105 (특장판)|아오야마 고쇼 (지은이)|서울미디어코믹스(서울문화사)|10000|9000|국내도서
복복깜짝북|미공개 (지은이)|복복서가|18500|16650|국내도서
원펀맨 One Punch Man 30|ONE (지은이), 무라타 유스케 (그림)|대원씨아이(만화)|5500|4950|국내도서
당신이 누군가를 죽였다|히가시노 게이고 (지은이), 최고은 (옮긴이)|북다|19800|17820|국내도서
너에게 들려주는 단단한 말|김종원 (지은이)|퍼스트펭귄|18000|16200|국내도서
불안 세대|조너선 하이트 (지은이), 이충호 (옮긴이)|웅진지식하우스|24800|22320|국내도서
더 좋은 문장을 쓰고 싶은 당신을 위한 필사책|이주윤 (지은이)|빅피시|22000|19800|국내도서
푸른 들판을 걷다|클레어 키건 (지은이), 허진 (옮긴이)|다산책방|16800|15120|국내도서
저속노화 식사법|정희원 (지은이)|테이스트북스|19000|17100|국내도서
ETS 토익 정기시험 기출문제집 1000 Vol. 4 Reading (리딩)|ETS (엮은이)|(주)YBM(와이비엠)|19800|17820|국내도서
명탐정 코난 105|아오야마 고쇼 (지은이)|서울미디어코믹스(서울문화사)|6000|5400|국내도서
ETS 토익 정기시험 기출문제집 1000 Vol. 4 Listening (리스닝)|ETS (엮은이)|